In [1]:
# bibliotecas usadas
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 

In [2]:
# para gerar o grafico no browser
%matplotlib inline

In [3]:
# local dos dados e modifica o diretorio
path='/home/ucfilho/Documents/Aulas/Ago_29_2018'
os.chdir(path)

In [4]:
# leitura dos dados
DAT=pd.read_csv("Hydrofoil_ago_29_2018.csv")
#DAT=pd.read_csv("Hydrofoil_ago_29_2018.csv",index_col="emit")

In [5]:
# separa as colunas de interesse
Index=[2,3,4,5,6,7]
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()

In [6]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [7]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [8]:
# separa os dados em normatizados e originais
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)

In [9]:
# separa os dados em treino e teste
train=[]
test=[]
train=DAT.sample(frac=0.8,random_state=None).copy() # poderia ser frac=0.5? sim ou nao?
test=DAT.drop(train.index).copy()
print(train.head())

    Hydrofoil    Froude  Resistence   Sinkage      Trim     Power
42   0.333333  0.728608    0.572917  0.758551  0.371806  0.359479
48   1.000000 -0.481417   -0.278646 -0.671829  0.719959 -0.812959
52   1.000000  0.210026   -0.015625  0.624976  0.511409 -0.404440
31   0.333333 -1.000000   -1.000000 -0.883085 -1.000000 -1.000000
11  -1.000000  0.210026    0.332812  0.151034  0.786861 -0.193631


In [10]:
# obtem as variaveis independentes (treino e teste)
x_train=train.iloc[:,[0,1]]
x_test=test.iloc[:,[0,1]]
X_OLD=DAT_OLD.iloc[:,[0,1]]

In [11]:
# obtem as variaveis dependentes (treino e teste)
Index=[2,3,4,5]
y_train=train.iloc[:,Index]
y_test=test.iloc[:,Index]
Y_OLD=DAT_OLD.iloc[:,Index]

In [12]:
def rede(i,j):
    clf = MLPRegressor(solver='lbfgs',activation='tanh',alpha=1e-5,
                       hidden_layer_sizes=(i, j),random_state=1)
    return clf

In [13]:
N=10
M=10
test=Original(y_test,Y_OLD)
resist_obs_test=[]
resist_obs_test=test['Resistence'].copy()
Melhor=1e99
ir=0
jr=0
for i in range(1,N):
    for j in range(1,M):
        col_names=list(y_train)
        clf=rede(i,j)
        clf.fit(x_train, y_train)
        y_calc_train=[]
        y_calc_test=[]
        y_calc_train=clf.predict(x_train)
        y_calc_test=clf.predict(x_test)
        y_calc_train=pd.DataFrame(y_calc_train)
        y_calc_test=pd.DataFrame(y_calc_test)
        y_calc_train.columns = col_names
        y_calc_test.columns = col_names
        y_calc_train=Original(y_calc_train,Y_OLD)
        y_calc_test=Original(y_calc_test,Y_OLD)
        resist_calc_test=[]
        resist_calc_test=y_calc_test['Resistence'].copy()
        mse=mean_squared_error(resist_obs_test,resist_calc_test)
        R2=r2_score(resist_obs_test,resist_calc_test)
        print("Neuronios=",i,j,"MSE teste=",mse,"R^2 teste=",R2)
        if(mse<Melhor):
            Melhor=mse.copy()
            ix=i
            jx=j
            R2r=R2
print("\n Melhor resposta:")
print("Neuronios=",ix,jx,"MSE teste=",Melhor,"R^2 teste=",R2r)

/home/ucfilho/.local/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Neuronios= 1 1 MSE teste= 506.60326307 R^2 teste= 0.962982450333
Neuronios= 1 2 MSE teste= 471.410446335 R^2 teste= 0.965553992872
Neuronios= 1 3 MSE teste= 219.559220302 R^2 teste= 0.983956786435
Neuronios= 1 4 MSE teste= 506.256370085 R^2 teste= 0.963007797837
Neuronios= 1 5 MSE teste= 255.923449309 R^2 teste= 0.981299648687
Neuronios= 1 6 MSE teste= 324.722895785 R^2 teste= 0.976272466447
Neuronios= 1 7 MSE teste= 347.179410668 R^2 teste= 0.974631566722
Neuronios= 1 8 MSE teste= 239.781594247 R^2 teste= 0.982479135605
Neuronios= 1 9 MSE teste= 414.088329957 R^2 teste= 0.969742525487
Neuronios= 2 1 MSE teste= 450.505858467 R^2 teste= 0.967081493139
Neuronios= 2 2 MSE teste= 506.615582265 R^2 teste= 0.962981550169
Neuronios= 2 3 MSE teste= 481.19543615 R^2 teste= 0.964839002716
Neuronios= 2 4 MSE teste= 147.343806226 R^2 teste= 0.98923357376
Neuronios= 2 5 MSE teste= 174.849760554 R^2 teste= 0.987223711004
Neuronios= 2 6 MSE teste= 129.987469465 R^2 teste= 0.990501802973
Neuronios= 2 